# Algorytm symulowanego wyżarzania (SA)

Grupa: Amelia Madej, Justyna Sarkowicz, Olga Sieradzan, Weronika Duda i Aleksandra Węgrzyn

## Biblioteki

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import itertools
import random
import math
from collections import deque
from openpyxl import load_workbook

## Pobór danych

In [ ]:
file_path1 = "./Dane_TSP_48.xlsx"
file_path2 = "./Dane_TSP_76.xlsx"
file_path3 = "./Dane_TSP_127.xlsx"

# index_col=0 zamienia pierwszą kolumne na indeksy wierszy 
# .to_numpy() zamienia ramkę danych na macierz
data1 = pd.read_excel(file_path1, index_col=0).to_numpy()
data2 = pd.read_excel(file_path2, index_col=0).to_numpy()
data3 = pd.read_excel(file_path3, index_col=0).to_numpy()

## Rodzaj ruchów

Poniżej znajduje się implementacja trzech rodzajów ruchów (generowania rozwiązań sąsiednich)

In [ ]:
# Wybieramy dwie pozycje i zmieniamy je miejscami 
def swap_move(path):  
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:]
        new_path[i], new_path[j] = new_path[j], new_path[i]
        yield new_path
 

# Odwracamy segment pomiędzy wybranymi wartościami czyli 
# [a b c d e] -> [a d c b e]
def two_opt_move(path):
    for i, j in itertools.combinations(range(len(path)), 2):
        new_path = path[:i] + path[i:j][::-1] + path[j:]
        yield new_path
 
# Wybieramy jeden wierzchołek w ścieżce, usuwamy z jednego miejsca i przerzucamy w inne miejsce
def insertion_move(path):
    n = len(path)
    for i in range(n):
        for j in range(n):
            if i != j:
                new_path = path[:]
                node = new_path.pop(i)
                new_path.insert(j, node)
                yield new_path


## Długość ścieżki

In [ ]:
# Funkcja obliczająca całkowity koszt ścieżki
# Suma odległości pomiędzy kolejnymi miastami oraz powrót do miasta początkowego

def calculate_path_cost(matrix, path):
    return sum(matrix[path[i - 1]][path[i]] for i in range(len(path))) + matrix[path[-1]][path[0]]

## Implementacja algorytmu

In [ ]:
def simulated_annealing(
    distance_matrix,
    initial_temperature=1000,
    cooling_rate=0.95,
    max_iterations=1000,
    min_temperature=1e-3,
    time_limit=None,
    move_type="swap"  
):
    start_time = time.time()
    n = len(distance_matrix)

    # Losowe wygenerowanie początkowej trasy
    initial_route = list(range(n))
    random.shuffle(initial_route)
    
    # Mapowanie typu ruchu na odpowiednią funkcję generatora sąsiadów
    move_generators = {
        "swap": swap_move,
        "two_opt": two_opt_move,
        "insertion": insertion_move
    }

    if move_type not in move_generators:
        raise ValueError(f"Nieznany typ ruchu: {move_type}")

    move_generator = move_generators[move_type]

    # Inicjalizacja bieżącego i najlepszego rozwiązania
    current_solution = initial_route
    current_cost = calculate_path_cost(distance_matrix, current_solution)
    best_solution = current_solution[:]
    best_cost = current_cost

    # Ustawienie początkowej temperatury
    temperature = initial_temperature

    for iteration in range(max_iterations):
        if time_limit and (time.time() - start_time > time_limit):
            print("Przekroczono limit czasu.")
            break
        
        # Sprawdzenie, czy temperatura osiągnęła wartość minimalną
        if temperature < min_temperature:
            print("Temperatura osiągnęła wartość minimalną.")
            break

        # Generowanie sąsiada na podstawie wybranego typu ruchu
        neighbors = list(move_generator(current_solution))
        neighbor = random.choice(neighbors)
        neighbor_cost = calculate_path_cost(distance_matrix, neighbor)

        
        # Różnica w kosztach między bieżącym a sąsiednim rozwiązaniem
        delta = neighbor_cost - current_cost

        if delta < 0:
            current_solution = neighbor
            current_cost = neighbor_cost
        else:
            # Akceptacja gorszego rozwiązania z prawdopodobieństwem zależnym od temperatury
            acceptance_probability = math.exp(-delta / temperature)
            if random.random() < acceptance_probability:
                current_solution = neighbor
                current_cost = neighbor_cost


        # Aktualizacja najlepszego rozwiązania, jeśli obecne jest lepsze
        if current_cost < best_cost:
            best_solution = current_solution[:]
            best_cost = current_cost

        # Schładzanie temperatury
        temperature *= cooling_rate

        if iteration % 100 == 0:
            print(f"Iteracja {iteration}, najlepszy koszt: {best_cost}, temperatura: {temperature:.2f}")

        total_time = time.time() - start_time

    return best_solution, best_cost, total_time

## Generowanie rozwiązań

Poniżej znajduje się zestawienie wyników z uwzględnieniem wpływu na wyniki różnych wartości parametrów algorytmu

Za podstawowe dane przyjmujemy :

* initial_temperature = 10000

* cooling_rate = 0.99

* max_iterations = 10000

* min_temperature = 0.00001

* time_limit = `None`

* move_type = `two_opt`

Następnie badane są wpływy zmian wartości poszczególnych parametrów 

### Metoda `swap`

In [ ]:
swap_p_1, swap_c_1, swap_t_1 = simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

swap_p_2, swap_c_2, swap_t_2 = simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

swap_p_3, swap_c_3, swap_t_3 = simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "swap"
    )

Przekroczono limit czasu (1000 s).


### Metoda `two_opt`

In [ ]:
opt_p_1, opt_c_1, opt_t_1 = simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

opt_p_2, opt_c_2, opt_t_2 = simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

opt_p_3, opt_c_3, opt_t_3 = simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )

Przekroczono limit czasu (1000 s).
Przekroczono limit czasu (1000 s).


### Metoda `insertion`

In [ ]:
ins_p_1, ins_c_1, ins_t_1 = simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

ins_p_2, ins_c_2, ins_t_2 = simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

ins_p_3, ins_c_3, ins_t_3 = simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "insertion"
    )

Układ w ramce danych

In [ ]:
W8 =  {
    "Długość ścieżki": [swap_c_1, opt_c_1, ins_c_1],
    "Ścieżka": [swap_p_1, opt_p_1, ins_p_1],
    "Czas": [swap_t_1, opt_t_1, ins_t_1]
}

sa1 = pd.DataFrame(data = W8)
sa1.index = ["Swap" , "2-opt", "Insertion"]

W9 =  {
    "Długość ścieżki": [swap_c_2, opt_c_2, ins_c_2],
    "Ścieżka": [swap_p_2, opt_p_2, ins_p_2],
    "Czas": [swap_t_2, opt_t_2, ins_t_2]
}

sa2 = pd.DataFrame(data = W9)
sa2.index = ["Swap" , "2-opt", "Insertion"]

W10 =  {
    "Długość ścieżki": [swap_c_3, opt_c_3, ins_c_3],
    "Ścieżka": [swap_p_3, opt_p_3, ins_p_3],
    "Czas": [swap_t_3, opt_t_3, ins_t_3]
}

sa3 = pd.DataFrame(data = W10)
sa3.index = ["Swap" , "2-opt", "Insertion"]

### Badanie wpływu zmian wartości parametru `initial_temperature`

In [ ]:
# DANE NR 1 
results = []
for initial_temperature in range(1000, 100000,  2000):
    best_path, best_cost, best_time = simulated_annealing(
        data1,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Konwersja wyników do DataFrame
tem1 = pd.DataFrame(results)

# DANE NR 2
results = []
for initial_temperature in range(1000, 100000,  2000):
    best_path, best_cost, best_time = simulated_annealing(
        data2,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Konwersja wyników do DataFrame
tem2 = pd.DataFrame(results)


# DANE NR 3
results = []
for initial_temperature in range(1000, 100000,  2000):
    best_path, best_cost, best_time = simulated_annealing(
        data3,
        initial_temperature= initial_temperature,
        cooling_rate=0.99,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": initial_temperature, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Konwersja wyników do DataFrame
tem3 = pd.DataFrame(results)

In [ ]:
# Połączenie wyników z trzech zbiorów danych w jeden DataFrame
merged_tem = tem1.merge(tem2, on="PARAMETR").merge(tem3, on="PARAMETR")

### Badanie wpływu zmian wartości parametru `cooling_rate`

In [ ]:
# DANE NR 1 
results = []
for cooling_rate in range(0.8, 0.999,  0.01):
    best_path, best_cost, best_time = simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})


cool1 = pd.DataFrame(results)


# DANE NR 2
results = []
for cooling_rate in range(0.8, 0.999,  0.01):
    best_path, best_cost, best_time = simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
cool2 = pd.DataFrame(results)


# DANE NR 3
results = []
for cooling_rate in range(0.8, 0.999,  0.01):
    best_path, best_cost, best_time = simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=cooling_rate,
        max_iterations=10000,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": cooling_rate, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
cool3 = pd.DataFrame(results)

In [ ]:
merged_cool = cool1.merge(cool2, on="PARAMETR").merge(cool3, on="PARAMETR")

### Badanie wpływu zmian wartości parametru `max_iterations`

In [ ]:
# DANE NR 1 
results = []
for max_iterations in range(100, 100000,  200):
    best_path, best_cost, best_timet = simulated_annealing(
        data1,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})


max1 = pd.DataFrame(results)


# DANE NR 2
results = []
for max_iterations in range(100, 100000,  200):
    best_path, best_cost, best_timet = simulated_annealing(
        data2,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
max2 = pd.DataFrame(results)


# DANE NR 3
results = []
for max_iterations in range(100, 100000,  200):
    best_path, best_cost, best_timet = simulated_annealing(
        data3,
        initial_temperature= 10000,
        cooling_rate=0.99,
        max_iterations=max_iterations,
        min_temperature=0.00001,
        time_limit= None,
        move_type= "two_opt"
    )
    results.append({"PARAMETR": max_iterations, "WYNIK": best_cost, "Ścieżka" : best_path, "CZAS": best_time})

# Tworzenie ramki danych
max3 = pd.DataFrame(results)

In [ ]:
merged_max = max1.merge(max2, on="PARAMETR").merge(max3, on="PARAMETR")

### Badanie wpływu zmian wartości parametru `min_temperature`

In [ ]:
# DANE NR 1
results = []
for min_temperature in np.logspace(-6, -2, num=10):  
    best_path, best_cost, best_timet = simulated_annealing(
        data1,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000, 
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost,  "Ścieżka" : best_path, "CZAS": best_time})

min_temp1 = pd.DataFrame(results)

# DANE NR 2
results = []
for min_temperature in np.logspace(-6, -2, num=10):
    best_path, best_cost, best_timet = simulated_annealing(
        data2,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000, 
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost,  "Ścieżka" : best_path, "CZAS": best_time})

min_temp2 = pd.DataFrame(results)

# DANE NR 3
results = []
for min_temperature in np.logspace(-6, -2, num=10):
    best_path, best_cost, best_timet = simulated_annealing(
        data3,
        initial_temperature=10000,
        cooling_rate=0.99,
        max_iterations=10000, 
        min_temperature=min_temperature,
        time_limit=None,
        move_type="two_opt"
    )
    results.append({"PARAMETR": min_temperature, "WYNIK": best_cost,  "Ścieżka" : best_path, "CZAS": best_time})

min_temp3 = pd.DataFrame(results)

In [ ]:
merged_min = min_temp1.merge(min_temp2, on="PARAMETR").merge(min_temp3, on="PARAMETR")

Zapis do pliku

In [ ]:
result = {
    "Porównanie_metod_1": sa1,
    "Porównanie_metod_2": sa2,
    "Porównanie_metod_3": sa3,
    "max_iterations": merged_max,
    "min_temp":  merged_min,
    "cooling_rate": merged_cool,
    "initial_temp": merged_tem
}

# Ścieżka do pliku Excel
file_name = "SA.xlsx"

# Zapisujemy dane do Excela
with pd.ExcelWriter(file_name) as writer:
    for sheet_name, df in result.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f"Wyniki zostały zapisane w pliku {file_name}")